## Save And Load PyTorch Model

In [1]:
import torch
import torch.nn as nn
import numpy as np
import time
from torch.utils.data import Dataset,DataLoader
from torchsummary import summary
from torch.optim import SGD
device='cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
x=[[1,2],[3,4],[5,6],[7,8]]
y=[[3],[7],[11],[15]]

In [3]:
class MyDataset(Dataset):
    def __init__(self,x,y):
        self.x=torch.tensor(x).float().to(device)
        self.y=torch.tensor(y).float().to(device)
    def __len__(self):
        return len(self.x)
    def __getitem__(self, ix):
        return self.x[ix],self.y[ix]

In [4]:
ds=MyDataset(x,y)
dl=DataLoader(ds,batch_size=2,shuffle=True)

In [5]:
model=nn.Sequential(
    nn.Linear(2,8),
    nn.ReLU(),
    nn.Linear(8,1)
).to(device)

In [6]:
summary(model,(1,2))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1, 8]              24
              ReLU-2                 [-1, 1, 8]               0
            Linear-3                 [-1, 1, 1]               9
Total params: 33
Trainable params: 33
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [7]:
loss_func=nn.MSELoss()
opt=SGD(model.parameters(),lr=0.001)
loss_history=[]
start=time.time()
for _ in range(50):
    for ix,iy in dl:
        opt.zero_grad()
        loss_value=loss_func(model(ix),iy)
        loss_value.backward()
        opt.step()
        loss_history.append(loss_value)
end=time.time()
print(end-start)

0.10435295104980469


In [8]:
save_path='mymodel.pth'
torch.save(model.state_dict(),save_path)

In [9]:
load_path='mymodel.pth'
model.load_state_dict(torch.load(load_path))

<All keys matched successfully>

In [10]:
val=[[8,9],[10,11],[1.5,2.5]]
val=torch.tensor(val).float().to(device)

In [11]:
model(val)

tensor([[16.8088],
        [20.6604],
        [ 4.2733]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [12]:
val.sum(-1)

tensor([17., 21.,  4.], device='cuda:0')